# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [27]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [28]:
from ipywidgets import *

In [29]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test


## Define dictionary

In [30]:
## Read ref_launch.yaml
# main_code_dir=config_dict[facility]['main_code_dir']
# dict_pars=config_dict[facility][img_size][run_type]
dict_pars={'nodes':10, 'queue':'regular','bins':'unenven','cores':64,'time':'0:30:00',
           'job_name':'3ptfnc',
            'suffix':'val_3d_64cube-sigma0.5','num_l':8,'img_slice':64,
                 }

dict_pars['staging_loc']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/'
dict_pars['code_dir']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test'
dict_pars['cores']=dict_pars['nodes']*64

In [31]:
### 3dgan 64cube
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-124_step-19390.npy'
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-118_step-18560.npy'


In [32]:
### 3dcgan 64cube
sigma=0.5
dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma)


In [33]:
# %%bash -s "$op_fldr"
# ls $1

In [34]:
dict_pars

{'nodes': 10,
 'queue': 'regular',
 'bins': 'unenven',
 'cores': 640,
 'time': '0:30:00',
 'job_name': '3ptfnc',
 'suffix': 'val_3d_64cube-sigma0.5',
 'num_l': 8,
 'img_slice': 64,
 'staging_loc': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/',
 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test',
 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy'}

In [35]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir={code_dir}
fname={fname}

srun -n {cores} python $code_dir/3pt_fcn_simple.py -n {num_l} --img_slice {img_slice} -idx $I -f $fname -sfx {suffix}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [36]:
fname='batch_3ptcn.sh'
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/batch_3ptcn.sh


In [37]:
## Move to staging locations in project space:
os.chdir(dict_pars['staging_loc'])

In [38]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=10
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=0:30:00
#SBATCH --job-name=3ptfnc

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test
fname=/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy

srun -n 640 python $code_dir/3pt_fcn_simple.py -n 8 --img_slice 64 -idx $I -f $fname -sfx val_3d_64cube-sigma0.5

echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [39]:
lst=np.arange(32)
# lst=[0,3,6,8,19]
lst2=[str(i) for i in lst]
strg=' '.join(lst2)
print(strg)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31


In [40]:
%%bash -s "$filename" "$strg" ## Use python variable in bash
# for i in $2; do echo $i; echo $1; done
for i in $2; do sbatch $1 $i; done

Submitted batch job 43100934
Submitted batch job 43100935
Submitted batch job 43100937
Submitted batch job 43100938
Submitted batch job 43100939
Submitted batch job 43100940
Submitted batch job 43100941
Submitted batch job 43100942
Submitted batch job 43100943
Submitted batch job 43100944
Submitted batch job 43100945
Submitted batch job 43100946
Submitted batch job 43100947
Submitted batch job 43100948
Submitted batch job 43100949
Submitted batch job 43100950
Submitted batch job 43100951
Submitted batch job 43100952
Submitted batch job 43100953
Submitted batch job 43100954
Submitted batch job 43100956
Submitted batch job 43100957
Submitted batch job 43100958
Submitted batch job 43100959
Submitted batch job 43100960
Submitted batch job 43100961
Submitted batch job 43100962
Submitted batch job 43100963
Submitted batch job 43100964
Submitted batch job 43100966
Submitted batch job 43100969
Submitted batch job 43100970
